# SageMaker Factorization Machine(FM) 및 KNN으로 추천 시스템 구축하기


**본 노트북은 김대근님의 노트북 내용을 많이 가지고 왔습니다.**<br>
기존 movielens 데이타를 에어라인 리뷰 데이터로 교체하고 이를 가공하는 부분을 추가 하였습니다.
알고리즘 설명 및 코드는 거의 유사합니다.
- https://github.com/daekeun-ml/recommendation-workshop/blob/master/0.Recommendation-System-FM-KNN.ipynb

*본 노트북 예제는 AWS 머신 러닝 블로그에 기고된 글들에 기반하여 SageMaker의 Factorization Machine(FM)으로 추천 시스템을 구축하는 방법을 설명합니다.*

References
- [Build a movie recommender with factorization machines on Amazon SageMaker](https://aws.amazon.com/ko/blogs/machine-learning/build-a-movie-recommender-with-factorization-machines-on-amazon-sagemaker/)
- [Amazon SageMaker Factorization Machines 알고리즘을 확장하여 추천 시스템 구현하기](https://aws.amazon.com/ko/blogs/korea/extending-amazon-sagemaker-factorization-machines-algorithm-to-predict-top-x-recommendations/)
- [Factorization Machine 논문](https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf)

#### 이 노트북은 아래와 같은 작업을 합니다.
1. FM 알고리즘 학습, 배포 및 추론
- 에어라인 리뷰 데이터 다운로드
- 데이터 전처리
    - 필요한 컬럼을 추출하여 interaction data set 형태로 만들기 (timestamp, user_id, item_id, rating)
    - 상호작용이 5개 이상인 유저만 추출
    - 유저, 아이템을 문자열에서 숫자로 변환
    - 데이터를 유저 기준으로 시간순으로 정렬한 후에 학습, 검증으로 8:2 로 분리
    - 학습의 인스턴스 * 컬럼의 갯수 대비 실제 사용한 데이터 영역을 알아 봅니다. (희소성, Sparsity)
    - 원핫 인코딩으로 희소 행렬 변환
        - 점수가 8점 이상이면 1, 미만이면 0으로 하여 y 레이블 생성
    - Protobuf 포맷으로 변환 후에 S3에 저장
- FM 학습 (이진 분류 문제)
    - Python SDK SageMaker2.0에 맞게 변경
- 모델 엔드포인트 배포
- 커스텀 시리얼라이즈 구현 (Python SDK SageMaker2.0에 맞게 변경맞게 신규 구현)

2.FM 모델 파라미터를 사용하여 KNN 학습, 배포 및 배치 추론
- FM 모델 아티펙트 다운로드
- 모델 파리미터 ($w_{0}, \mathbf{w}, \mathbf{V}$)를 추출
- 학습/추론 데이터를 아래와 같이 새롭게 구성
    - Item latent 행렬: k-NN 모델 학습에 사용; $a_i = concat(V, \; w)$
    - User latent 행렬: 추론에 사용; $a_u = concat(V, \; 1)$
- KNN 모델 학습
- 배치 추론
- 에어라인 Top-K 추천





# 1. 에어라인 데이터셋으로 FM 모델 훈련 및 배포하기
---

MovieLens 데이터의 user_id, item_id가 숫자로 이미 학습에 바로 사용가능한 데이터입니다.
하지만, 에어라인 데이터는 모두 문자열로 데이타가 주어졌습니다. <br>
이 데이타를 labelencoder를 통해서 숫자로 바꾸어서 여기서 사용합니다. <br>
본 예제에서는 사용자가 5번 이상의 리뷰를 남긴 사용자만을 대상으로 하여 총 748명의 사용자와 293개의 에어라인 대해 1부터 10까지의 등급이 부여된 데이터를 사용합니다.

### Factorization Machine
---

### 개요

일반적인 추천 문제들은 user가 행, item이 열, rating이 값으로 이루어진 행렬을 데이터셋으로 하여 Matrix Factorization 기법을 활용하는데, real-world의 다양한 메타데이터 피처(feature)들을 그대로 적용하기에는 어려움이 있습니다. Factoriztion Machine(이하 FM) 알고리즘은 Matrix Factorization의 개념을 확장하여 메타데이터 피처들을 같이 고려하고 피처 간의 상호 관계(interaction)를 선형 계산 복잡도로 자동으로 모델링할 수 있기에, 피처 엔지니어링에 들어가는 노력을 크게 줄일 수 있습니다.

### 설명

다양한 메타데이터 피처를 고려하기 위해 아래 그림처럼 user와 item을 원-핫 인코딩으로 변환하고 추가 피처들을 그대로 concatenate하여 `f(user, item, additional features) = rating` 형태의 선형 회귀(Linear Regression) 문제로 변환하여 풀 수 있습니다. 

![Factorization Machine](./img/fm.png "Factorization Machine")

하지만, 추천 문제를 선형 회귀로만 풀려고 하면 피처 간의 상호 관계를 고려할 수 없기에 아래 수식처럼 피처 간의 상호 관계를 모델링하는 항을 추가하여 다항 회귀(Polynomial Regression)로 변환해야 합니다.

$$
\begin{align} \hat{y}(\mathbf{x}) = w_{0} + \sum_{i=1}^{d} w_{i} x_{i} + \sum_{i=1}^d \sum_{j=i+1}^d x_{i} x_{j} w_{ij}, \;\; x \in \mathbb{R}^d \tag {1} 
\end{align}
$$ 
$d$는 피처 갯수로, $x \in \mathbb{R}^d$는 단일 샘플의 피처 벡터를 나타냅니다.

하지만 대부분의 추천 시스템 데이터셋은 희소하기에(sparse) cold-start 문제가 있으며, 추가적으로 고려해야 하는 피처들이 많아질 수록 계산이 매우 복잡해집니다. (예: user가 6만명, item 갯수가 5천개, 추가 피처가 5천개일 경우 70,000x70,000 행렬을 예측해야 합니다.)  

FM은 이러한 문제들을 행렬 분해 기법을 활용하여 feature 쌍(예: user, item) 간의 상호 관계를 내적(dot product)으로 변환하고 
수식을 재구성하여 계산 복잡도를 $O(kd^2)$에서 $O(kd)$로 감소시켰습니다. (수식 (2)에서 추가적인 계산을 거치면 계산 복잡도를 선형으로 감소할 수 있습니다. 자세한 내용은 논문을 참조하세요.) 

$$
\begin{align}
\hat{y}(\mathbf{x}) = w_{0} + \sum_{i=1}^{d} w_i x_i + \sum_{i=1}^d\sum_{j=i+1}^d  x_{i} x_{j} \langle\mathbf{v}_i, \mathbf{v}_j\rangle \tag{2} 
\end{align}
$$

$$
\begin{align}
\langle \textbf{v}_i , \textbf{v}_{j} \rangle = \sum_{f=1}^k v_{i,f} v_{j,f},\; k: \text{dimension of latent feature}  \tag{3}
\end{align}
$$

위의 모델을 2-way(degree = 2) FM이라고 하며, 이를 일반화한 d-way FM도 있지만, 보통 2-way FM를 많이 사용합니다. SageMaker의 FM 또한 2-way FM입니다.

FM이 훈련하는 파라메터 튜플은 ($w_{0}, \mathbf{w}, \mathbf{V}$) 이며, 의미는 아래와 같습니다.
- $w_{0} \in \mathbb{R}$: global bias
- $\mathbf{w} \in \mathbb{R}^d$: 피처 벡터 $\mathbf{x}_i$의 가중치
- $\mathbf{V} \in \mathbb{R}^{n \times k}$: 피처 임베딩 행렬로 i번째 행은 $\mathbf{v}_i$


FM은 위의 수식에서 알 수 있듯이 closed form이며 시간 복잡도가 선형이기 때문에, 다수의 user & item과 메타데이터들이 많은 추천 문제에 적합합니다.
훈련 방법은 대표적으로 Gradient Descent, ALS(Alternating Least Square), MCMC(Markov Chain Monte Carlo)가 있으며, AWS에서는 이 중 딥러닝 아키텍처에 기반한 Gradient Descent를 MXNet 프레임워크를 이용하여 훈련합니다.

In [1]:
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
# from sagemaker.predictor import json_deserializer
from sagemaker.deserializers import JSONDeserializer
# from sagemaker.amazon.amazon_estimator import get_image_uri
import numpy as np
from scipy.sparse import lil_matrix
import pandas as pd
import boto3, io, os, csv, json

## 에어라인 리뷰 데이터 다운로드

데이터는 비행기 에어라인의 리뷰에 대한 데이타 셋 입니다. <br>
평가에 대한 데이터가 안락함, 청결, 음료, 음식, 화장실, 직원 서비스, 종합에 대한 평가가 있습니다. 여기서는 "종합"에 대한 평가 점수를 사용합니다.
- Skytrax User Reviews Dataset (August 2nd, 2015)
    - https://github.com/quankiquanki/skytrax-reviews-dataset

In [2]:
import os
data_dir = "airlines_data"
os.makedirs(data_dir, exist_ok=True)
!cd $data_dir && wget https://raw.githubusercontent.com/quankiquanki/skytrax-reviews-dataset/master/data/airline.csv

--2020-10-03 09:21:10--  https://raw.githubusercontent.com/quankiquanki/skytrax-reviews-dataset/master/data/airline.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.228.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.228.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34752262 (33M) [text/plain]
Saving to: ‘airline.csv.3’

airline.csv.3       100%[===================>]  33.14M  16.9MB/s    in 2.0s    

2020-10-03 09:21:15 (16.9 MB/s) - ‘airline.csv.3’ saved [34752262/34752262]



In [3]:
import pandas as pd
airline_df = pd.read_csv(data_dir + '/airline.csv', parse_dates=['date'])
print("airline_df: ", airline_df.shape)
airline_df.head()

airline_df:  (41396, 20)


,airline_name,link,title,author,author_country,date,content,aircraft,type_traveller,cabin_flown,route,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,wifi_connectivity_rating,value_money_rating,recommended
0,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,D Ito,Germany,2015-04-10,Outbound flight FRA/PRN A319. 2 hours 10 min f...,NaN,NaN,Economy,NaN,7.0,4.0,4.0,4.0,0.0,NaN,NaN,4.0,1
1,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Ron Kuhlmann,United States,2015-01-05,Two short hops ZRH-LJU and LJU-VIE. Very fast ...,NaN,NaN,Business Class,NaN,10.0,4.0,5.0,4.0,1.0,NaN,NaN,5.0,1
2,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,E Albin,Switzerland,2014-09-14,Flew Zurich-Ljubljana on JP365 newish CRJ900. ...,NaN,NaN,Economy,NaN,9.0,5.0,5.0,4.0,0.0,NaN,NaN,5.0,1
3,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Tercon Bojan,Singapore,2014-09-06,Adria serves this 100 min flight from Ljubljan...,NaN,NaN,Business Class,NaN,8.0,4.0,4.0,3.0,1.0,NaN,NaN,4.0,1
4,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,L James,Poland,2014-06-16,WAW-SKJ Economy. No free snacks or drinks on t...,NaN,NaN,Economy,NaN,4.0,4.0,2.0,1.0,2.0,NaN,NaN,2.0,0


### 필요한 데이타 컬럼을 추출하여 Interaction Data Set으로 사용하기

다운로드 받은 데이터에서 'date','airline_name', 'author', 'overall_rating' 만을 선택하여 사용 합니다. 
또한 아래와 같이 컬럼 이름을 변경 합니다.
- airline_name --> item_id
- author --> user_id
- overall_rating --> rating


In [4]:
def get_interaction_data(airline_df):
    '''
    필요한 컬럼만 선별하고, 컬럼 이름 변경
    '''
    a_interactions_df = airline_df.copy()
    a_interactions_df = a_interactions_df[['date','airline_name', 'author', 'overall_rating']]
    a_interactions_df['author'] = a_interactions_df['author'].str.replace(" ","")
    a_interactions_df.rename(columns = {'airline_name':'item_id', 'author':'user_id',
                                   'overall_rating': 'rating'}, inplace = True) 
    print("a_interactions_df: ",a_interactions_df.shape)
    return a_interactions_df

a_interactions_df = get_interaction_data(airline_df)
a_interactions_df.head()

a_interactions_df:  (41396, 4)


,date,item_id,user_id,rating
0,2015-04-10,adria-airways,DIto,7.0
1,2015-01-05,adria-airways,RonKuhlmann,10.0
2,2014-09-14,adria-airways,EAlbin,9.0
3,2014-09-06,adria-airways,TerconBojan,8.0
4,2014-06-16,adria-airways,LJames,4.0


#### 유저 선별
널 데이터 제거, 유저별로 5개 이상의 인터렉션이 있는 유저만 선별 합니다.

In [5]:
def clean_data(air_df, user_limit=5):
    '''
    널 데이터 제거, 유저별로 5개 이상의 인터렉션이 있는 유저만 선별
    '''
    df = air_df.copy()
    df = df.dropna() # 널인 데이타는 제거
    user_g = df.groupby('user_id').count() # 유저 별로 집계

    u5_list = user_g[user_g.item_id >= user_limit].index # 5개의 상호작용 유저만 선택
    df = df[df.user_id.isin(u5_list)] # 5명의 유저만 선택
    # air_df.groupby('user_id').count().sort_values(by='item_id', ascending=False)
    
    return df

air_df = clean_data(a_interactions_df)


#### 유저 및 아이템에 대해서 숫자로 변환

- 레이블 인코더를 이용하여 숫자로 변환
- 사용된 레이블 인코더는 추후에 id-->실제 이름 으로 변경하기 위해서 다시 사용 됩니다.


In [6]:
from sklearn import preprocessing
def make_labelencoder(a_interactions_df):
    '''
    user, item에 대해서 레이블 인코더 생성
    '''
    le_user = preprocessing.LabelEncoder()
    le_item = preprocessing.LabelEncoder()
    le_user.fit(a_interactions_df.user_id)
    le_item.fit(a_interactions_df.item_id)
    print(f"num_user: {len(le_user.classes_)}")
    print(f"num_item: {len(le_item.classes_)}")
    return le_user, le_item

le_user, le_item = make_labelencoder(air_df)

def format_interaction_tb(air_df,le_user,le_item ):
    df = air_df.copy()
    user_id = le_user.transform(df.user_id)
    item_id = le_item.transform(df.item_id)    
    en_df = pd.DataFrame({'date': air_df.date, 'user_id':user_id, 'item_id':item_id, 
                          'rating':air_df.rating.astype(int)})
    
    return en_df

encode_df = format_interaction_tb(air_df,le_user,le_item )   


num_user: 748
num_item: 293


#### 데이타를 Train(학습), Test (검증) 를 시간순으로 8:2 로 구분

한 명의 유저에 대해서 Train, Test를 시간순으로 8:2 를 구분 합니다.<br>
예로서 특정 유저가 5개의 추천이 있으면, 시간순으로 소팅하여 앞의 4개는 Train, 뒤의 1개는 Test에 속합니다.<br>
Train, Test의 Unique User ID는 동일하게 748명 입니다.

In [7]:
pd.options.display.max_rows = 5
def split_holdout(data, pct):
    '''
    시간순으로 pct 비율 만큼 학습,테스트 데이터 셋으로 구분
    '''
    df = data.copy()
    # Rank per each subgroup, 'USER_ID'
    ranks = df.groupby('user_id').date.rank(pct=True, method='first')
    df = df.join((ranks> pct).to_frame('holdout'))
    
    df = df.drop('date', axis=1)
    
    holdout = df[df['holdout']].drop('holdout', axis=1)
    holdout = holdout.sample(frac=1, random_state = 100)
    train = df[~df['holdout']].drop('holdout', axis=1)    
    train = train.sample(frac=1, random_state = 100)    
    
    return train, holdout

user_airline_ratings_train, user_aireline_ratings_test = split_holdout(encode_df, pct=0.8)

In [8]:
print("train unique user_id # : ", user_airline_ratings_train.user_id.nunique())
print("test unique user_id # :", user_aireline_ratings_test.user_id.nunique())
print(f"train shape: {user_airline_ratings_train.shape}")
print(f"test shape: {user_aireline_ratings_test.shape}")

train unique user_id # :  748
test unique user_id # : 748
train shape: (4968, 3)
test shape: (1577, 3)


### Sparsity(희소성) 비율 구하기

학습 데이터는 총 4,968 개 입니다. 이 데이터가 왜 희소한지 궁금할 수 있습니다. 전체 매트릭스가 5,161,752 에서 9,936 만을 사용하므로 희소성은 99,8% 입니다.
- 피쳐수: 1,039
    - 사용자와 292개의 에어라인 회사 (747 + 292 = 1,039)
- 학습 인스터스 수: 4,968
- 총 매트릭스 수: 1,039 * 4,968 = 5,161,752
- 총 매트릭스 중에 사용된 셀 수: 4,968 * 2 = 9,936 (2는 사용자에 1개, 아이템에 1개를 사용하기에 2 입니다.)
- 매트릭스 희소성(Sparsity): 9,936 / 5,161,752 = 0.0019 (99.8%)

In [9]:
nb_users = user_airline_ratings_train['user_id'].max()
nb_movies = user_airline_ratings_train['item_id'].max()
nb_features = nb_users + nb_movies

nb_ratings_train = len(user_airline_ratings_train.index)
nb_ratings_test = len(user_aireline_ratings_test.index)

total_matrix_size = nb_features * nb_ratings_train
filled_matrix_size =  nb_ratings_train * 2 # 2 means that 1 is for user and 1 is for item

print("# of users: {}".format(nb_users))
print("# of airlines: {}".format(nb_movies))
print("Training Count: {}".format(nb_ratings_train))
print("Test Count: {}".format(nb_ratings_test))
print("Features (# of users + # of movies): {}".format(nb_features))
print("Sparsity: {:.6}%".format(str((1 - filled_matrix_size / total_matrix_size) * 100)))

# of users: 747
# of airlines: 292
Training Count: 4968
Test Count: 1577
Features (# of users + # of movies): 1039
Sparsity: 99.807%


#### 원-핫 인코딩 희소 행렬 변환

FM의 입력 데이터 포맷인 원-핫 인코딩 희소 행렬로 변환하겠습니다. 물론 희소 행렬이 아닌 밀집(dense) 행렬도 가능하지만, 데이터가 많아질수록 계산 속도가 느려지므로, 희소 행렬을 추천합니다. 

참고로, 에어라인 데이터셋은 별도의 메타데이터 피처가 존재하지 않아 747명의 사용자와 292개 에어라인에 대해서만 원-핫 인코딩 변환을 수행하므로 변환 후 피처의 차원은 747+292=1,039입니다.

**또한, 본 예시에서는 rating이 8 이상인 에어라인들에 대한 이진 분류 문제로 간소화합니다. (즉, rating이 8 이상일 경우 y = 1, 8 미만일 경우 y = 0 입니다.)**

아래 셀은 약 20초 소요되며, 변환 후 데이터셋의 차원은 rating 개수 x 피쳐 개수 입니다.

In [10]:
pd.options.display.max_rows = 10
user_airline_ratings_train.rating.describe() # 중간값이 8 임. 8 이상을 긍정, 이하를 부정으로 간주 함

count    4968.000000
mean        7.090781
std         2.485475
min         1.000000
25%         6.000000
50%         8.000000
75%         9.000000
max        10.000000
Name: rating, dtype: float64

In [11]:
%%time
def loadDataset(df, lines, columns):
    
    # 피처는 원-핫 인코딩 희소 행렬로 변환합니다.
    X = lil_matrix((lines, columns)).astype('float32')
    Y = []
    line = 0
    for line, (index, row) in enumerate(df.iterrows()):
            X[line,row['user_id']-1] = 1
            X[line, nb_users+(row['item_id']-1)] = 1
            
            # Y lable 생성
            if int(row['rating']) >= 8:
                Y.append(1) # 긍정
            else:
                Y.append(0) # 부정

    Y = np.array(Y).astype('float32')            
    return X,Y

X_train, Y_train = loadDataset(user_airline_ratings_train, nb_ratings_train, nb_features)
X_test, Y_test = loadDataset(user_aireline_ratings_test, nb_ratings_test, nb_features)

CPU times: user 745 ms, sys: 0 ns, total: 745 ms
Wall time: 744 ms


학습 및 검증의 전체 Shape를 확인하고, y의 1 과 0 분포를 확인 합니다.

In [12]:
print(X_train.shape)
print(Y_train.shape)
assert X_train.shape == (nb_ratings_train, nb_features)
assert Y_train.shape == (nb_ratings_train, )
zero_labels = np.count_nonzero(Y_train)
print("Training labels: {} zeros, {} ones".format(zero_labels, nb_ratings_train-zero_labels))

print(X_test.shape)
print(Y_test.shape)
assert X_test.shape  == (nb_ratings_test, nb_features)
assert Y_test.shape  == (nb_ratings_test, )
zero_labels = np.count_nonzero(Y_test)
print("Test labels: {} zeros, {} ones".format(zero_labels, nb_ratings_test-zero_labels))

(4968, 1039)
(4968,)
Training labels: 2718 zeros, 2250 ones
(1577, 1039)
(1577,)
Test labels: 869 zeros, 708 ones


#### Protobuf 포맷 변환 후 S3에 저장

In [13]:
import sagemaker
bucket = sagemaker.Session().default_bucket()
#bucket = '[YOUR-BUCKET]'
prefix = 'fm-hol'

if bucket.strip() == '':
    raise RuntimeError("bucket name is empty.")

train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test')

output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)

아래 셀은 약 15초 소요됩니다.

In [14]:
%%time
def writeDatasetToProtobuf(X, bucket, prefix, key, d_type, Y=None):
    buf = io.BytesIO()
    if d_type == "sparse":
        smac.write_spmatrix_to_sparse_tensor(buf, X, labels=Y)
    else:
        smac.write_numpy_to_dense_tensor(buf, X, labels=Y)
        
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
    
fm_train_data_path = writeDatasetToProtobuf(X_train, bucket, train_prefix, train_key, "sparse", Y_train)    
fm_test_data_path  = writeDatasetToProtobuf(X_test, bucket, test_prefix, test_key, "sparse", Y_test)    
  
print("Training data S3 path: ", fm_train_data_path)
print("Test data S3 path: ", fm_test_data_path)
print("FM model output S3 path: ", output_prefix)

Training data S3 path:  s3://sagemaker-ap-northeast-2-057716757052/fm-hol/train/train.protobuf
Test data S3 path:  s3://sagemaker-ap-northeast-2-057716757052/fm-hol/test/test.protobuf
FM model output S3 path:  s3://sagemaker-ap-northeast-2-057716757052/fm-hol/output
CPU times: user 577 ms, sys: 34.2 ms, total: 611 ms
Wall time: 1.77 s


## FM 학습

본 핸즈온은 하이퍼파라메터 튜닝 없이 휴리스틱한 하이퍼파라메터들을 사용합니다. 

- `feature_dim`: 피처 개수로 본 핸즈온에서는 1.039으로 설정해야 합니다.
- `mini_batch_size`: 본 핸즈온에서는 256 으로 설정합니다.
- `num_factors`: latent factor 차원으로 본 핸즈온에서는 64차원으로 설정합니다.
- `epochs`: 본 핸즈온에서는 100 에폭으로 설정합니다.

In [15]:
# 세이지 메이커 신규 버전 다운로드
!pip install --upgrade sagemaker

Requirement already up-to-date: sagemaker in /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages (2.13.0)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_p36/bin/python -m pip install --upgrade pip' command.


In [16]:
from sagemaker import image_uris, session
fm_image = image_uris.retrieve("factorization-machines", session.Session().boto_region_name, version="latest")

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


이제 훈련을 시작하기 위한 모든 준비가 완료되었으며, 여러분께서는 `fit()` 메소드만 호출하면 됩니다. <br>
훈련은 약 4분에서 5분이 소요되며(순수 훈련에 소요되는 시간은 훨씬 짧지만, 훈련 인스턴스를 프로비저닝하는 시간이 고정적으로 소요됩니다), <br>
검증 데이터셋에 대한 accuracy는 약 66%에 F1 스코어는 약 0.71입니다. (아래 output 메세지 참조)

```
[10/03/2020 09:24:01 INFO 140262587922240] #quality_metric: host=algo-1, test binary_classification_accuracy <score>=0.667089410273
[10/03/2020 09:24:01 INFO 140262587922240] #quality_metric: host=algo-1, test binary_classification_cross_entropy <loss>=0.61242604422
[10/03/2020 09:24:01 INFO 140262587922240] #quality_metric: host=algo-1, test binary_f_1.000 <score>=0.711696869852
```

In [17]:
instance_type_training = 'ml.c4.xlarge'
fm = sagemaker.estimator.Estimator(image_uri = fm_image,
                                   role = get_execution_role(), 
                                   instance_count=1, 
                                   instance_type=instance_type_training,
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session(),
                                  
                                  )

fm.set_hyperparameters(feature_dim=nb_features,
                      predictor_type='binary_classifier',
                      mini_batch_size=256,
                      num_factors=64,
                      epochs=100)

In [18]:
%%time
fm.fit({'train': fm_train_data_path, 'test': fm_test_data_path})

2020-10-03 09:21:22 Starting - Starting the training job...
2020-10-03 09:21:23 Starting - Launching requested ML instances...
2020-10-03 09:22:21 Starting - Preparing the instances for training......
2020-10-03 09:23:18 Downloading - Downloading input data
2020-10-03 09:23:18 Training - Downloading the training image...
2020-10-03 09:23:51 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python2.7/site-packages/pandas/util/nosetester.py:13: DeprecationWarning: Importing from numpy.testing.nosetester is deprecated, import from numpy.testing instead.
  from numpy.testing import nosetester
[10/03/2020 09:23:53 INFO 140262587922240] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers': u'auto'

## 모델 엔드포인트 배포
배포 또한, 매우 간단하게 `deploy()` 메소드로 수행하실 수 있습니다. 배포는 약 5분에서 10분이 소요됩니다.

In [19]:
%%time
instance_type_inference = 'ml.m5.large'
fm_predictor = fm.deploy(instance_type=instance_type_inference, initial_instance_count=1)

-----------!CPU times: user 178 ms, sys: 5.19 ms, total: 183 ms
Wall time: 5min 31s


## 엔드포인트 추론

일부 테스트로 10개의 인스턴스에 대해서 Rating이 1 인지 0인지를 예측 해보았습니다.
예제로 아래의 결과를 보면 10개 중에 8개의 예측 정확도를 보입니다.
아래의 결과는 실행시에 하이퍼 파라미터의 값에 따라서 다르게 나올 수 있습니다.
```
prediction labels:
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]
true list:
 [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
Prediction Accuracy:
 [True, False, True, True, True, True, True, False, True, True]
```

### 커스텀 시리얼라이져 구현

아래 URL을 참조하여 구현 하였습니다.
- https://github.com/aws/sagemaker-python-sdk/blob/8fff159389aa63941ccf0c59567c3eb7936a2c62/src/sagemaker/serializers.py    

In [32]:

class CustomJSONSerializer(sagemaker.serializers.BaseSerializer):
    """
    # Sample Code:
    # https://github.com/aws/sagemaker-python-sdk/blob/8fff159389aa63941ccf0c59567c3eb7936a2c62/src/sagemaker/serializers.py    
    # How to test
        js = CustomJSONSerializer()
        sample = X_test[1000:1001].toarray()
        print(js.serialize(sample))    
    """

    CONTENT_TYPE = "application/json"

    def serialize(self, data):
        """Serialize data of various formats to a JSON formatted string.
        Args:
            data (object): Data to be serialized.
        Returns:
            str: The data serialized as a JSON string.
        """
        
        if isinstance(data, np.ndarray):
            js = {'instances': []}
            for row in data:
                js['instances'].append({'features': row.tolist()})

            
            return json.dumps(js)
        else:
            print("Not np.ndarray type")
            return json.dumps(data)


fm_predictor.serializer = CustomJSONSerializer()
JSONDeserializer.ACCEPT = 'application/json'
fm_predictor.deserializer = JSONDeserializer()




In [33]:
result = fm_predictor.predict(X_test[1000:1010].toarray())
pred_labels = list()
for pred in result['predictions']:
    label = pred['predicted_label']
    pred_labels.append(label)

true_list = Y_test[1000:1010].tolist()
judge = [p == t for p, t in zip (pred_labels,true_list)]

print("true list:\n", true_list)
print("prediction labels:\n", pred_labels)
print("Prediction Accuracy:\n", judge)

true list:
 [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
prediction labels:
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]
Prediction Accuracy:
 [True, False, True, True, True, True, True, False, True, True]



**지금까지 기본적인 사용법을 알아보았으며, 여기에서 실습을 종료하셔도 됩니다. 실습을 일찍 끝내셨거나, 좀 더 깊은 내용을 원하신다면 아래 셀들을 순차적으로 실행해 주세요.
[주의] 실시간 예측을 제공하기 위해 엔드포인트를 계속 실행할 필요가 없는 경우, 과금을 막기 위해 엔드포인트를 삭제해 주세요.**
<br>

# 2. (Optional) top-k 추천을 위하여 FM 모델의 모델 파라메터를 사용하여 knn으로 훈련 및 배포하기
---

이제 SageMaker에 모델을 작성하고 저장 했으므로 동일한 FM 모델을 다운로드하여 KNN 모델에 맞게 다시 패키지 할 수 있습니다. 

### 모델 아티팩트 다운로드

In [34]:
#!pip install mxnet # 필요한 경우 주석을 해제하여 mxnet을 설치해 주세요
import mxnet as mx 
model_file_name = "model.tar.gz"
model_full_path = fm.output_path + "/" + fm.latest_training_job.job_name + "/output/" + model_file_name
print("Model Path: ", model_full_path)

# FM 모델 아티팩트(model.tar.gz) 다운로드 
os.system("aws s3 cp " + model_full_path+ " .")

# 모델 아티팩트 압축 해제
os.system("tar xzvf " + model_file_name)
os.system("unzip -o model_algo-1")
os.system("mv symbol.json model-symbol.json")
os.system("mv params model-0000.params")

Model Path:  s3://sagemaker-ap-northeast-2-057716757052/fm-hol/output/factorization-machines-2020-10-03-09-21-21-934/output/model.tar.gz


0

### 모델 데이터 분리

FM에서 훈련한 파라메터 튜플 ($w_{0}, \mathbf{w}, \mathbf{V}$)을 가져옵니다.

In [35]:
# 모델 추출
m = mx.module.Module.load('./model', 0, False, label_names=['out_label'])
V = m._arg_params['v'].asnumpy() # 2625 x 64
w = m._arg_params['w1_weight'].asnumpy() # 2625 x1
b = m._arg_params['w0_weight'].asnumpy() # 1
print(V.shape, w.shape, b.shape)

(1039, 64) (1039, 1) (1,)


### 데이터셋 재가공

이제 FM 모델에서 추출한 모델 파라메터를 다시 패키지하여 k-NN 모델을 훈련하기 위한 준비를 수행해 보겠습니다.
이 프로세스는 두 개의 데이터셋을 생성합니다.

- Item latent 행렬: k-NN 모델 학습에 사용; $a_i = concat(V, \; w)$
- User latent 행렬: 추론에 사용; $a_u = concat(V, \; 1)$

참고로, 본 핸즈온 코드는 user 및 item ID가 있는 시나리오에만 적용됩니다. 그러나 실제 데이터에는 추가 메타데이터(예: user의 경우 나이, 우편번호, 성별이 포함되고 영화의 경우 영화 장르, 주요 키워드)들이 포함될 수 있습니다. 이러한 경우에는 아래 방법으로 user 및 item 벡터를 추출할 수 있습니다.

- item과 item feature를 $x_i$로 인코딩 후 $\mathbf{V}, \mathbf{w}$에 내적; $a_i = concat(V^T \cdot x_i , \; w^T \cdot x_i)$
- user와 user feature를 $x_u$로 인코딩 후 $\mathbf{V}$에 내적; $a_u = concat(V^T \cdot x_u, \; , 1)$

$a_i$를 사용하여 k-NN 모델을 훈련하고 $a_u$를 사용하여 추론을 수행하시면 됩니다.

In [36]:
# item latent matrix - concat(V[i], w[i]).  
knn_item_matrix = np.concatenate((V[nb_users:], w[nb_users:]), axis=1) # 292 x 65
knn_train_label = np.arange(1,nb_movies+1) # [1, 2, 3, ..., 291, 292]

# user latent matrix - concat (V[u], 1) 
ones = np.ones(nb_users).reshape((nb_users, 1)) # 747x1
knn_user_matrix = np.concatenate((V[:nb_users], ones), axis=1) # 747 x 65

### k-NN 모델 훈련

k-NN 모델은 기본 index_type (faiss.Flat)을 사용합니다.  대규모 데이터셋의 경우 속도가 느려지기에, 이런 경우 더 빠른 훈련을 위해 다른 index_type 매개 변수를 사용할 수 있습니다. index 유형에 대한 자세한 내용은 k-NN 설명서를 참조해 주세요.

In [37]:
print('KNN train features shape = ', knn_item_matrix.shape)
knn_prefix = 'knn'
knn_output_prefix  = 's3://{}/{}/output'.format(bucket, knn_prefix)
knn_train_data_path = writeDatasetToProtobuf(knn_item_matrix, bucket, knn_prefix, train_key, "dense", knn_train_label)
print('Uploaded KNN train data: {}'.format(knn_train_data_path))

KNN train features shape =  (292, 65)
Uploaded KNN train data: s3://sagemaker-ap-northeast-2-057716757052/knn/train.protobuf


In [38]:
nb_recommendations = 100
knn_image = image_uris.retrieve("knn", session.Session().boto_region_name, version="latest")

knn = sagemaker.estimator.Estimator(knn_image,
    get_execution_role(),
    instance_count=1,
    instance_type=instance_type_training,
    output_path=knn_output_prefix,
    sagemaker_session=sagemaker.Session())

knn.set_hyperparameters(feature_dim=knn_item_matrix.shape[1], k=nb_recommendations, 
                        index_metric="INNER_PRODUCT", predictor_type='classifier', sample_size=200000)
fit_input = {'train': knn_train_data_path}

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


훈련을 시작합니다. 아래 셀의 수행 시간은 약 4분에서 5분이 소요됩니다.

In [39]:
%%time
knn.fit(fit_input)
knn_model_name =  knn.latest_training_job.job_name
print("Created model: ", knn_model_name)

2020-10-03 09:46:33 Starting - Starting the training job...
2020-10-03 09:46:35 Starting - Launching requested ML instances...
2020-10-03 09:47:32 Starting - Preparing the instances for training......
2020-10-03 09:48:15 Downloading - Downloading input data...
2020-10-03 09:48:37 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[10/03/2020 09:49:13 INFO 139974427436864] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'index_metric': u'L2', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'_log_level': u'info', u'feature_dim': u'auto', u'faiss_index_ivf_nlists': u'auto', u'epochs': u'1', u'index_type': u'faiss.Flat', u'_faiss_index_nprobe': u'5', u'_kvstore': u'dist_async', u'_num_kv_servers': u'1', u'mini_batch_size': u'5000'}
[10/03/2020 09:49:13 INFO 139974427436864] Merging with provided configuration from /opt/ml/in

배치 추론에서 참조할 수 있도록 모델을 생성합니다.

In [40]:
# 다음 단계에서 배치 추론 중에 참조할 수 있도록 모델 저장
sm = boto3.client(service_name='sagemaker')
primary_container = {
    'Image': knn.image_uri,
    'ModelDataUrl': knn.model_data,
}

knn_model = sm.create_model(
        ModelName = knn.latest_training_job.job_name,
        ExecutionRoleArn = knn.role,
        PrimaryContainer = primary_container)

### Batch Transform

Amazon SageMaker의 Batch Transform 기능을 사용하면 대규모로 배치 추론 결과를 생성할 수 있습니다. <br>
아래 셀의 실행이 완료되기까지는 약 4분 소요됩니다.

In [41]:
%%time
# 추론 데이터 S3에 업로드
knn_batch_data_path = writeDatasetToProtobuf(knn_user_matrix, bucket, knn_prefix, train_key, "dense")
print("Batch inference data path: ", knn_batch_data_path)

# Transformer 객체 초기화
transformer = sagemaker.transformer.Transformer(
    base_transform_job_name="knn",
    model_name=knn_model_name,
    instance_count=1,
    instance_type=instance_type_inference,
    output_path=knn_output_prefix,
    accept="application/jsonlines; verbose=true"
)

# 변환 작업 시작
transformer.transform(knn_batch_data_path, content_type='application/x-recordio-protobuf')
transformer.wait()

# S3에서 출력 파일 다운로드
results_file_name = "inference_output"
inference_output_file = "knn/output/train.protobuf.out"
s3_client = boto3.client('s3')
s3_client.download_file(bucket, inference_output_file, results_file_name)
with open(results_file_name) as f:
    results = f.readlines()  

Batch inference data path:  s3://sagemaker-ap-northeast-2-057716757052/knn/train.protobuf
............................2020-10-03T09:54:16.127:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
Docker entrypoint called with argument(s): serve
Running default environment configuration script
[10/03/2020 09:54:15 INFO 140511654475584] loaded entry point class algorithm.serve.server_config:config_api
[10/03/2020 09:54:15 INFO 140511654475584] loading entry points
[10/03/2020 09:54:15 INFO 140511654475584] loaded request iterator text/csv
[10/03/2020 09:54:15 INFO 140511654475584] loaded request iterator application/x-recordio-protobuf
[10/03/2020 09:54:15 INFO 140511654475584] loaded request iterator application/json
[10/03/2020 09:54:15 INFO 140511654475584] loaded request iterator application/jsonlines
[10/03/2020 09:54:15 INFO 140511654475584] loaded response encoder application/x-recordio-protobuf
[10/03/2020 09:54:15 INFO 140511654475584] loaded 

## 에어라인 추천 (top-k 추론 예시)

아래는 90번 사용자인 BobMotto에 대해서 추천된 에어라인 이름을 보여주고 있습니다.
'airline_dist'는 airline의 유사도를 의미합니다. 

In [42]:
import json
pd.options.display.max_rows = 20
test_user_idx = 89 # 인덱스는 0부터 시작하므로 90번 사용자의 인덱스는 89입니다.
u_one_json = json.loads(results[test_user_idx])

airline_id_list = [int(airline_id) for airline_id in u_one_json['labels']]
airline_name_list = [le_item.inverse_transform([airline_name])[0] for airline_name in airline_id_list]
airline_dist_list = [round(distance, 4) for distance in u_one_json['distances']]

recommend_df = pd.DataFrame({'airline_id': airline_id_list, 
                             'airline_name': airline_name_list, 
                             'airline_dist': airline_dist_list})
user_name = le_user.inverse_transform([test_user_idx])[0]
print("Recommendations for user: ", user_name)
recommend_df.head(30)

Recommendations for user:  BobMotto


,airline_id,airline_name,airline_dist
0,45,aircalin,0.1268
1,241,sun-express,0.1386
2,133,hawaiian-airlines,0.1576
3,42,airasia-x,0.1673
4,179,mango,0.1757
...,...,...,...
25,291,yangon-airways,0.2760
26,192,olympic-air,0.2770
27,10,air-arabia,0.2820
28,244,swiss-international-air-lines,0.2833
